# 🌾 GrassClover-Style Generation with Stable Diffusion

Este notebook implementa geração de imagens sintéticas de pastagens brasileiras usando **Stable Diffusion**, seguindo o estilo visual do **GrassClover Dataset** (Skovsen et al., CVPR 2019).

## 🎯 Objetivos:
- Gerar imagens **top-down** de pastagens com Stable Diffusion
- Adaptar para **gramíneas brasileiras** (Brachiaria, Panicum, Cynodon)
- Seguir **metodologia GrassClover** (densidade, perspectiva, resolução)
- **Ultra-compatível** com Google Colab (debugging extensivo)

## 📚 Referência:
- **Paper**: Skovsen et al. "The GrassClover Image Dataset for Semantic and Hierarchical Species Understanding in Agriculture" (CVPR Workshops, 2019)
- **Adaptação**: Espécies temperadas → Tropicais brasileiras

---

## 🚀 Setup Ultra-Compatível para Colab

**IMPORTANTE**: Este notebook foi desenvolvido para máxima compatibilidade com Google Colab Free/Pro.

In [ ]:
# 🔍 VERIFICAÇÃO INICIAL DO AMBIENTE
print("=" * 60)
print("🌾 GRASSCLOVER STABLE DIFFUSION GENERATOR")
print("=" * 60)

import sys
import platform
from datetime import datetime

print(f"📅 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python: {sys.version.split()[0]}")
print(f"💻 Platform: {platform.system()} {platform.release()}")
print(f"📍 Working Directory: {sys.path[0] if sys.path else 'Unknown'}")

# Detectar se estamos no Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔥 Ambiente: Google Colab (DETECTADO)")
except ImportError:
    IN_COLAB = False
    print("💻 Ambiente: Local/Jupyter (DETECTADO)")

print("\n" + "=" * 60)

In [ ]:
# 🧠 VERIFICAÇÃO E SETUP DE GPU
print("🔍 Verificando disponibilidade de GPU...\n")

# Tentar importar torch primeiro
try:
    import torch
    TORCH_AVAILABLE = True
    print(f"✅ PyTorch: {torch.__version__}")
except ImportError:
    TORCH_AVAILABLE = False
    print("❌ PyTorch não encontrado - será instalado")

# Verificar CUDA se PyTorch disponível
if TORCH_AVAILABLE:
    cuda_available = torch.cuda.is_available()
    print(f"🔥 CUDA disponível: {cuda_available}")
    
    if cuda_available:
        gpu_count = torch.cuda.device_count()
        print(f"🚀 Número de GPUs: {gpu_count}")
        
        for i in range(gpu_count):
            gpu_name = torch.cuda.get_device_name(i)
            gpu_memory = torch.cuda.get_device_properties(i).total_memory
            gpu_memory_gb = gpu_memory / (1024**3)
            print(f"  GPU {i}: {gpu_name}")
            print(f"  Memória: {gpu_memory_gb:.1f} GB")
        
        # Configurar device
        device = torch.device("cuda")
        print(f"\n🎯 Usando device: {device}")
        
        # Limpeza inicial de memória
        torch.cuda.empty_cache()
        print("🧹 Cache GPU limpo")
        
    else:
        device = torch.device("cpu")
        print(f"\n⚠️  GPU não disponível - usando CPU")
        print("💡 Para habilitar GPU no Colab: Runtime > Change runtime type > GPU")
else:
    device = None
    print("\n⏳ PyTorch será instalado na próxima célula")

print("\n" + "=" * 60)

In [ ]:
# 📦 INSTALAÇÃO DE DEPENDÊNCIAS ESSENCIAIS
print("📦 Instalando dependências essenciais...\n")

# Lista de pacotes essenciais
essential_packages = [
    "torch",
    "torchvision", 
    "diffusers",
    "transformers",
    "accelerate",
    "pillow",
    "matplotlib",
    "numpy"
]

# Função para instalar com debug
def install_package(package_name, quiet=True):
    """Instala pacote com debugging"""
    import subprocess
    
    print(f"⏳ Instalando {package_name}...")
    
    try:
        cmd = ["pip", "install", package_name]
        if quiet:
            cmd.append("--quiet")
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        if result.returncode == 0:
            print(f"✅ {package_name} instalado com sucesso")
            return True
        else:
            print(f"❌ Erro ao instalar {package_name}:")
            print(f"STDOUT: {result.stdout}")
            print(f"STDERR: {result.stderr}")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"⏰ Timeout ao instalar {package_name}")
        return False
    except Exception as e:
        print(f"💥 Exceção ao instalar {package_name}: {e}")
        return False

# Instalar apenas se necessário
for package in essential_packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✅ {package} já disponível")
    except ImportError:
        success = install_package(package)
        if not success:
            print(f"🚨 FALHA CRÍTICA: Não foi possível instalar {package}")
            print(f"📋 DEBUG INFO para copy/paste:")
            print(f"Package: {package}")
            print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")
            print(f"Python: {sys.version}")
            break

print(f"\n🎉 Instalação concluída!")
print("=" * 60)

In [ ]:
# 🔄 REIMPORTAÇÃO E VERIFICAÇÃO FINAL
print("🔄 Verificando importações finais...\n")

# Imports essenciais com debugging
try:
    import torch
    import torchvision.transforms as transforms
    print(f"✅ PyTorch: {torch.__version__}")
    
    # Reconfigurar device após instalação
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        print(f"🚀 GPU: {gpu_name} ({gpu_memory:.1f} GB)")
    else:
        device = torch.device("cpu")
        print("💻 Device: CPU")
        
except ImportError as e:
    print(f"❌ Erro PyTorch: {e}")
    device = None

try:
    from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
    print(f"✅ Diffusers importado")
    DIFFUSERS_AVAILABLE = True
except ImportError as e:
    print(f"❌ Erro Diffusers: {e}")
    DIFFUSERS_AVAILABLE = False

try:
    from PIL import Image, ImageEnhance, ImageFilter
    import matplotlib.pyplot as plt
    import numpy as np
    print(f"✅ PIL, Matplotlib, NumPy importados")
    BASIC_LIBS_AVAILABLE = True
except ImportError as e:
    print(f"❌ Erro bibliotecas básicas: {e}")
    BASIC_LIBS_AVAILABLE = False

# Status final
ALL_READY = device is not None and DIFFUSERS_AVAILABLE and BASIC_LIBS_AVAILABLE

print(f"\n{'🎯 SISTEMA PRONTO!' if ALL_READY else '🚨 PROBLEMAS DETECTADOS!'}")
if not ALL_READY:
    print("\n📋 DEBUG COPY/PASTE INFO:")
    print(f"Device: {device}")
    print(f"Diffusers: {DIFFUSERS_AVAILABLE}")
    print(f"Basic libs: {BASIC_LIBS_AVAILABLE}")
    print(f"In Colab: {IN_COLAB}")

print("=" * 60)

## 🎨 Pipeline Stable Diffusion para GrassClover

Configuração do pipeline otimizado para geração de pastagens no estilo GrassClover.

In [ ]:
# 🎨 CONFIGURAÇÃO DO PIPELINE STABLE DIFFUSION
print("🎨 Configurando Stable Diffusion Pipeline...\n")

# Parâmetros do pipeline
MODEL_ID = "runwayml/stable-diffusion-v1-5"  # Modelo base confiável
TORCH_DTYPE = torch.float16 if device.type == "cuda" else torch.float32
LOW_CPU_MEM_USAGE = True
ENABLE_ATTENTION_SLICING = True

print(f"📦 Modelo: {MODEL_ID}")
print(f"🔢 Dtype: {TORCH_DTYPE}")
print(f"💾 Low CPU mem: {LOW_CPU_MEM_USAGE}")
print(f"⚡ Attention slicing: {ENABLE_ATTENTION_SLICING}")

# Função para carregar pipeline com debugging
def load_stable_diffusion_pipeline():
    """Carrega pipeline com máximo debugging"""
    try:
        print("⏳ Carregando modelo...")
        
        # Carregar pipeline
        pipe = StableDiffusionPipeline.from_pretrained(
            MODEL_ID,
            torch_dtype=TORCH_DTYPE,
            low_cpu_mem_usage=LOW_CPU_MEM_USAGE,
            use_safetensors=True
        )
        
        print("✅ Pipeline carregado")
        
        # Mover para device
        pipe = pipe.to(device)
        print(f"🚀 Pipeline movido para {device}")
        
        # Otimizações
        if ENABLE_ATTENTION_SLICING:
            pipe.enable_attention_slicing()
            print("⚡ Attention slicing habilitado")
            
        # Scheduler otimizado
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
        print("🔧 Scheduler otimizado (DPMSolver)")
        
        # Verificação de memória
        if device.type == "cuda":
            torch.cuda.empty_cache()
            allocated = torch.cuda.memory_allocated() / (1024**3)
            cached = torch.cuda.memory_reserved() / (1024**3)
            print(f"💾 GPU Memory - Allocated: {allocated:.2f}GB, Cached: {cached:.2f}GB")
        
        print("\n✅ Pipeline configurado com sucesso!")
        return pipe
        
    except Exception as e:
        print(f"💥 ERRO ao carregar pipeline: {e}")
        print(f"\n📋 DEBUG COPY/PASTE:")
        print(f"Model: {MODEL_ID}")
        print(f"Device: {device}")
        print(f"Dtype: {TORCH_DTYPE}")
        print(f"Error: {type(e).__name__}: {e}")
        return None

# Carregar pipeline
if ALL_READY:
    pipe = load_stable_diffusion_pipeline()
    PIPELINE_READY = pipe is not None
else:
    pipe = None
    PIPELINE_READY = False

print("=" * 60)

## 🌾 Prompts para Pastagens Brasileiras - Estilo GrassClover

Prompts específicos para gerar pastagens tropicais seguindo a metodologia visual do GrassClover.

In [ ]:
# 🌾 DEFINIÇÃO DE PROMPTS GRASSCLOVER-STYLE
print("🌾 Configurando prompts para pastagens brasileiras...\n")

# Prompts base seguindo metodologia GrassClover
GRASSCLOVER_PROMPTS = {
    'brachiaria_dominant': {
        'positive': (
            "aerial top-down view of dense brazilian brachiaria grass pasture, "
            "scientific photography, high resolution, natural lighting, "
            "brachiaria brizantha, thick green grass blades, dense coverage, "
            "tropical pasture, agricultural field, bird's eye view, "
            "detailed grass texture, realistic, photorealistic, "
            "ground sampling distance 4-8 pixels per millimeter, "
            "green pasture, cattle grazing land, forage grass"
        ),
        'negative': (
            "people, animals, buildings, vehicles, roads, fence, "
            "side view, human perspective, low angle, "
            "artificial, cartoon, painting, drawing, "
            "blurry, low quality, watermark, text, "
            "dead grass, brown grass, desert, sand"
        ),
        'description': "Brachiaria dominante - pastagem densa"
    },
    
    'panicum_lush': {
        'positive': (
            "overhead drone view of panicum maximum grass field, "
            "mombaça grass, tanzânia grass, tall tropical grass, "
            "lush green pasture, aerial perspective, "
            "scientific agricultural photography, detailed, "
            "high quality, natural sunlight, "
            "dense vegetation, forage quality, "
            "top-down view, bird's eye perspective"
        ),
        'negative': (
            "ground level, eye level, side angle, "
            "people, animals, machinery, infrastructure, "
            "artistic, stylized, painted, sketch, "
            "low resolution, pixelated, compressed, "
            "autumn, winter, dry season, yellow grass"
        ),
        'description': "Panicum exuberante - gramínea alta"
    },
    
    'mixed_pasture': {
        'positive': (
            "aerial view mixed tropical pasture, brachiaria and panicum, "
            "diverse grass species, natural grass composition, "
            "overhead agricultural photography, "
            "brazilian tropical grassland, cattle pasture, "
            "varied grass heights, natural diversity, "
            "realistic lighting, high detail, "
            "top-down perspective, scientific quality"
        ),
        'negative': (
            "uniform, monotonous, artificial lawn, "
            "human view, ground perspective, "
            "decorative, ornamental, urban grass, "
            "low quality, blurred, distorted, "
            "buildings, roads, people, animals"
        ),
        'description': "Pastagem mista - diversidade de espécies"
    },
    
    'cynodon_dense': {
        'positive': (
            "dense cynodon grass field from above, tifton grass, "
            "coast-cross grass, aerial photography, "
            "thick carpet-like grass coverage, "
            "uniform green pasture, top-down view, "
            "high quality agricultural image, "
            "natural lighting, detailed texture, "
            "scientific photography, realistic"
        ),
        'negative': (
            "sparse, patchy, uneven coverage, "
            "side view, angled view, ground level, "
            "artistic, stylized, non-photographic, "
            "poor quality, low resolution, "
            "weeds, bare soil, brown patches"
        ),
        'description': "Cynodon denso - cobertura uniforme"
    },
    
    'early_growth': {
        'positive': (
            "young grass pasture aerial view, early growth stage, "
            "emerging brachiaria seedlings, sparse coverage, "
            "some visible soil between grass, "
            "top-down agricultural photography, "
            "natural lighting, realistic, "
            "establishing pasture, new growth, "
            "scientific quality, high detail"
        ),
        'negative': (
            "mature, fully established, dense coverage, "
            "ground perspective, side angle, "
            "artificial, painted, artistic, "
            "poor lighting, low quality, "
            "dead vegetation, dried grass"
        ),
        'description': "Crescimento inicial - cobertura esparsa"
    }
}

# Parâmetros de geração
GENERATION_PARAMS = {
    'width': 512,
    'height': 512, 
    'num_inference_steps': 25,  # Balanceio qualidade/velocidade
    'guidance_scale': 7.5,      # Aderência ao prompt
    'num_images_per_prompt': 1,
    'eta': 0.0,                 # Determinismo
    'generator_seed': 42        # Reprodutibilidade inicial
}

print(f"📝 {len(GRASSCLOVER_PROMPTS)} prompts configurados:")
for key, prompt_data in GRASSCLOVER_PROMPTS.items():
    print(f"  • {key}: {prompt_data['description']}")
    
print(f"\n⚙️  Parâmetros de geração:")
for param, value in GENERATION_PARAMS.items():
    print(f"  • {param}: {value}")
    
print("\n✅ Prompts configurados!")
print("=" * 60)

## 🎯 Função de Geração com Debugging Completo

In [ ]:
# 🎯 FUNÇÃO DE GERAÇÃO COM DEBUGGING EXTENSIVO
print("🎯 Configurando função de geração...\n")

def generate_grassclover_image(prompt_key, custom_seed=None, debug=True):
    """
    Gera imagem no estilo GrassClover com debugging completo
    
    Args:
        prompt_key: Chave do prompt (ex: 'brachiaria_dominant')
        custom_seed: Seed personalizada (opcional)
        debug: Ativar prints de debug
    
    Returns:
        dict com imagem e metadados
    """
    
    if not PIPELINE_READY:
        print("❌ Pipeline não está pronto!")
        return None
    
    if prompt_key not in GRASSCLOVER_PROMPTS:
        print(f"❌ Prompt key '{prompt_key}' não encontrada!")
        print(f"Chaves disponíveis: {list(GRASSCLOVER_PROMPTS.keys())}")
        return None
    
    try:
        # Configurar seed
        seed = custom_seed if custom_seed is not None else GENERATION_PARAMS['generator_seed']
        generator = torch.Generator(device=device).manual_seed(seed)
        
        # Obter prompts
        prompt_data = GRASSCLOVER_PROMPTS[prompt_key]
        positive_prompt = prompt_data['positive']
        negative_prompt = prompt_data['negative']
        
        if debug:
            print(f"🌾 Gerando: {prompt_data['description']}")
            print(f"🎲 Seed: {seed}")
            print(f"📝 Prompt: {positive_prompt[:100]}...")
            print(f"❌ Negative: {negative_prompt[:50]}...")
            
            # Monitoramento de memória inicial
            if device.type == "cuda":
                torch.cuda.empty_cache()
                mem_before = torch.cuda.memory_allocated() / (1024**3)
                print(f"💾 GPU Memory antes: {mem_before:.2f}GB")
        
        # Geração
        print("⏳ Iniciando geração...")
        
        with torch.autocast(device.type):
            result = pipe(
                prompt=positive_prompt,
                negative_prompt=negative_prompt,
                width=GENERATION_PARAMS['width'],
                height=GENERATION_PARAMS['height'],
                num_inference_steps=GENERATION_PARAMS['num_inference_steps'],
                guidance_scale=GENERATION_PARAMS['guidance_scale'],
                num_images_per_prompt=GENERATION_PARAMS['num_images_per_prompt'],
                eta=GENERATION_PARAMS['eta'],
                generator=generator
            )
        
        if debug:
            print("✅ Geração concluída!")
            
            # Monitoramento de memória final
            if device.type == "cuda":
                mem_after = torch.cuda.memory_allocated() / (1024**3)
                print(f"💾 GPU Memory depois: {mem_after:.2f}GB")
                print(f"📊 Diferença: {mem_after - mem_before:.2f}GB")
        
        # Extrair imagem
        image = result.images[0]
        
        # Metadados
        metadata = {
            'prompt_key': prompt_key,
            'description': prompt_data['description'],
            'seed': seed,
            'generation_params': GENERATION_PARAMS.copy(),
            'timestamp': datetime.now().isoformat(),
            'model_id': MODEL_ID,
            'device': str(device)
        }
        
        return {
            'image': image,
            'metadata': metadata,
            'success': True
        }
        
    except Exception as e:
        print(f"💥 ERRO na geração: {e}")
        print(f"\n📋 DEBUG COPY/PASTE:")
        print(f"Prompt key: {prompt_key}")
        print(f"Seed: {seed}")
        print(f"Device: {device}")
        print(f"Pipeline ready: {PIPELINE_READY}")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {e}")
        
        # Limpeza de emergência
        if device.type == "cuda":
            torch.cuda.empty_cache()
            print("🧹 Cache GPU limpo (emergência)")
        
        return {
            'image': None,
            'metadata': None,
            'success': False,
            'error': str(e)
        }

# Função para visualização
def display_generation_result(result, show_metadata=True):
    """Exibe resultado da geração com metadados"""
    if not result['success']:
        print(f"❌ Geração falhou: {result.get('error', 'Erro desconhecido')}")
        return
    
    # Exibir imagem
    plt.figure(figsize=(10, 10))
    plt.imshow(result['image'])
    plt.axis('off')
    
    # Título com informações
    metadata = result['metadata']
    title = f"{metadata['description']}\nSeed: {metadata['seed']} | {metadata['generation_params']['width']}x{metadata['generation_params']['height']}"
    plt.title(title, fontsize=14, pad=20)
    
    plt.tight_layout()
    plt.show()
    
    # Metadados detalhados
    if show_metadata:
        print(f"\n📊 Metadados da Geração:")
        print(f"  • Tipo: {metadata['description']}")
        print(f"  • Seed: {metadata['seed']}")
        print(f"  • Resolução: {metadata['generation_params']['width']}x{metadata['generation_params']['height']}")
        print(f"  • Steps: {metadata['generation_params']['num_inference_steps']}")
        print(f"  • Guidance: {metadata['generation_params']['guidance_scale']}")
        print(f"  • Modelo: {metadata['model_id']}")
        print(f"  • Device: {metadata['device']}")
        print(f"  • Timestamp: {metadata['timestamp']}")

print("✅ Funções de geração configuradas!")
print("=" * 60)

## 🧪 Teste Inicial - Uma Imagem de Cada Tipo

In [ ]:
# 🧪 TESTE INICIAL - GERAR UMA IMAGEM DE CADA TIPO
print("🧪 Iniciando teste de geração...\n")

if PIPELINE_READY:
    # Selecionar alguns prompts para teste
    test_prompts = ['brachiaria_dominant', 'mixed_pasture', 'cynodon_dense']
    test_results = []
    
    print(f"🎯 Gerando {len(test_prompts)} imagens de teste...\n")
    
    for i, prompt_key in enumerate(test_prompts, 1):
        print(f"\n{'='*40}")
        print(f"🌾 TESTE {i}/{len(test_prompts)}: {prompt_key}")
        print(f"{'='*40}")
        
        # Gerar imagem
        result = generate_grassclover_image(
            prompt_key=prompt_key,
            custom_seed=42 + i,  # Seed diferente para cada teste
            debug=True
        )
        
        if result['success']:
            print(f"✅ Sucesso!")
            test_results.append(result)
            
            # Exibir resultado
            display_generation_result(result)
        else:
            print(f"❌ Falha na geração!")
            break
        
        # Limpeza entre gerações
        if device.type == "cuda":
            torch.cuda.empty_cache()
    
    print(f"\n🎉 Teste concluído! {len(test_results)}/{len(test_prompts)} imagens geradas com sucesso.")
    
else:
    print("❌ Pipeline não está pronto - não é possível executar testes")
    print("\n🔧 Verifique as células anteriores para resolver problemas de configuração")

print("=" * 60)

## 🎨 Pós-processamento Estilo GrassClover

Ajustes para deixar as imagens mais próximas do estilo visual do GrassClover Dataset.

In [ ]:
# 🎨 PÓS-PROCESSAMENTO ESTILO GRASSCLOVER
print("🎨 Configurando pós-processamento GrassClover...\n")

def grassclover_postprocess(image, intensity=1.0, debug=False):
    """
    Aplica pós-processamento para aproximar do estilo GrassClover
    
    Args:
        image: PIL Image
        intensity: Intensidade dos ajustes (0.0-2.0)
        debug: Mostrar etapas do processamento
    
    Returns:
        PIL Image processada
    """
    
    try:
        if debug:
            print(f"🎨 Iniciando pós-processamento (intensidade: {intensity})")
        
        # Cópia para não modificar original
        processed = image.copy()
        
        # 1. Ajuste de contraste (GrassClover tem contraste marcante)
        contrast_factor = 1.0 + (0.3 * intensity)
        enhancer = ImageEnhance.Contrast(processed)
        processed = enhancer.enhance(contrast_factor)
        
        if debug:
            print(f"  ✅ Contraste ajustado: {contrast_factor:.2f}")
        
        # 2. Ajuste de saturação (verdes mais vívidos)
        saturation_factor = 1.0 + (0.2 * intensity)
        enhancer = ImageEnhance.Color(processed)
        processed = enhancer.enhance(saturation_factor)
        
        if debug:
            print(f"  ✅ Saturação ajustada: {saturation_factor:.2f}")
        
        # 3. Sharpening sutil (detalhes de textura)
        if intensity > 0.5:
            sharpness_factor = 1.0 + (0.1 * intensity)
            enhancer = ImageEnhance.Sharpness(processed)
            processed = enhancer.enhance(sharpness_factor)
            
            if debug:
                print(f"  ✅ Nitidez ajustada: {sharpness_factor:.2f}")
        
        # 4. Slight blur para simular imperfeições naturais
        if intensity > 0.3:
            blur_radius = 0.3 * intensity
            processed = processed.filter(ImageFilter.GaussianBlur(radius=blur_radius))
            
            if debug:
                print(f"  ✅ Blur natural aplicado: {blur_radius:.2f}px")
        
        # 5. Ajuste de brilho (simular condições de campo)
        brightness_factor = 1.0 + (0.05 * intensity * (np.random.random() - 0.5))
        enhancer = ImageEnhance.Brightness(processed)
        processed = enhancer.enhance(brightness_factor)
        
        if debug:
            print(f"  ✅ Brilho ajustado: {brightness_factor:.2f}")
            print(f"🎉 Pós-processamento concluído!")
        
        return processed
        
    except Exception as e:
        print(f"💥 Erro no pós-processamento: {e}")
        print(f"📋 DEBUG COPY/PASTE:")
        print(f"Image mode: {image.mode if image else 'None'}")
        print(f"Image size: {image.size if image else 'None'}")
        print(f"Intensity: {intensity}")
        print(f"Error: {type(e).__name__}: {e}")
        return image  # Retornar original se falhar

def compare_before_after(original, processed, title="Comparação"):
    """
    Exibe comparação lado a lado
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    
    axes[0].imshow(original)
    axes[0].set_title("Original (Stable Diffusion)", fontsize=12)
    axes[0].axis('off')
    
    axes[1].imshow(processed)
    axes[1].set_title("Processado (Estilo GrassClover)", fontsize=12)
    axes[1].axis('off')
    
    fig.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

print("✅ Funções de pós-processamento configuradas!")
print("=" * 60)

In [ ]:
# 🧪 TESTE DO PÓS-PROCESSAMENTO
print("🧪 Testando pós-processamento GrassClover...\n")

if PIPELINE_READY and 'test_results' in locals() and test_results:
    # Usar primeira imagem dos testes anteriores
    test_image_data = test_results[0]
    original_image = test_image_data['image']
    
    print(f"🎨 Aplicando pós-processamento em: {test_image_data['metadata']['description']}")
    
    # Aplicar pós-processamento
    processed_image = grassclover_postprocess(
        image=original_image,
        intensity=1.2,  # Intensidade média-alta
        debug=True
    )
    
    # Exibir comparação
    compare_before_after(
        original=original_image,
        processed=processed_image,
        title=f"Pós-processamento: {test_image_data['metadata']['description']}"
    )
    
    print("✅ Teste de pós-processamento concluído!")
    
else:
    print("⚠️  Nenhuma imagem de teste disponível para pós-processamento")
    print("Execute primeiro a célula de teste de geração")

print("=" * 60)

## 🚀 Geração em Lote com Seeds Diferentes

Gera múltiplas variações de pastagens para criar um dataset diversificado.

In [ ]:
# 🚀 GERAÇÃO EM LOTE - DATASET DIVERSIFICADO
print("🚀 Configurando geração em lote...\n")

def generate_grassclover_batch(num_images=6, apply_postprocess=True, debug=True):
    """
    Gera lote de imagens variadas estilo GrassClover
    
    Args:
        num_images: Número total de imagens
        apply_postprocess: Aplicar pós-processamento
        debug: Debugging detalhado
        
    Returns:
        Lista de resultados
    """
    
    if not PIPELINE_READY:
        print("❌ Pipeline não está pronto!")
        return []
    
    # Distribuir tipos de pastagem
    prompt_keys = list(GRASSCLOVER_PROMPTS.keys())
    batch_results = []
    
    print(f"🎯 Gerando {num_images} imagens com {len(prompt_keys)} tipos de pastagem...\n")
    
    for i in range(num_images):
        # Selecionar tipo de pastagem (rotação)
        prompt_key = prompt_keys[i % len(prompt_keys)]
        
        # Seed única para cada imagem
        seed = 42 + i * 100 + np.random.randint(0, 50)
        
        print(f"\n{'='*50}")
        print(f"🌾 IMAGEM {i+1}/{num_images}: {prompt_key}")
        print(f"🎲 Seed: {seed}")
        print(f"{'='*50}")
        
        try:
            # Gerar imagem base
            result = generate_grassclover_image(
                prompt_key=prompt_key,
                custom_seed=seed,
                debug=debug
            )
            
            if result['success']:
                # Aplicar pós-processamento se solicitado
                if apply_postprocess:
                    processed_image = grassclover_postprocess(
                        image=result['image'],
                        intensity=np.random.uniform(0.8, 1.4),  # Variação aleatória
                        debug=False
                    )
                    
                    # Atualizar resultado
                    result['processed_image'] = processed_image
                    result['metadata']['postprocessed'] = True
                    
                    if debug:
                        print("🎨 Pós-processamento aplicado")
                
                # Adicionar índice
                result['metadata']['batch_index'] = i
                result['metadata']['total_batch'] = num_images
                
                batch_results.append(result)
                print(f"✅ Sucesso! ({len(batch_results)}/{num_images})")
                
            else:
                print(f"❌ Falha na geração da imagem {i+1}")
                
        except Exception as e:
            print(f"💥 Erro na imagem {i+1}: {e}")
            if debug:
                print(f"Prompt key: {prompt_key}")
                print(f"Seed: {seed}")
        
        # Limpeza entre gerações
        if device.type == "cuda":
            torch.cuda.empty_cache()
    
    print(f"\n🎉 Lote concluído: {len(batch_results)}/{num_images} imagens geradas!")
    return batch_results

def display_batch_results(batch_results, max_display=6):
    """
    Exibe resultados do lote em grid
    """
    if not batch_results:
        print("❌ Nenhum resultado para exibir")
        return
    
    # Limitar exibição
    results_to_show = batch_results[:max_display]
    n_images = len(results_to_show)
    
    # Calcular grid
    cols = min(3, n_images)
    rows = (n_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
    
    # Garantir que axes seja sempre 2D
    if rows == 1:
        axes = axes.reshape(1, -1)
    if cols == 1:
        axes = axes.reshape(-1, 1)
    
    for i, result in enumerate(results_to_show):
        row = i // cols
        col = i % cols
        
        # Usar imagem processada se disponível
        image = result.get('processed_image', result['image'])
        
        axes[row, col].imshow(image)
        axes[row, col].axis('off')
        
        # Título com informações
        metadata = result['metadata']
        title = f"{metadata['description']}\nSeed: {metadata['seed']}"
        axes[row, col].set_title(title, fontsize=10)
    
    # Ocultar eixos não usados
    for i in range(n_images, rows * cols):
        row = i // cols
        col = i % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"🌾 Dataset GrassClover Brasileiro - {n_images} Imagens", 
                fontsize=16, fontweight='bold', y=0.98)
    plt.show()

print("✅ Funções de geração em lote configuradas!")
print("=" * 60)

In [ ]:
# 🚀 EXECUTAR GERAÇÃO EM LOTE
print("🚀 Iniciando geração em lote...\n")

if PIPELINE_READY:
    # Configurações do lote
    BATCH_SIZE = 6  # Número total de imagens
    APPLY_POSTPROCESS = True
    
    print(f"⚙️  Configurações:")
    print(f"  • Imagens: {BATCH_SIZE}")
    print(f"  • Pós-processamento: {APPLY_POSTPROCESS}")
    print(f"  • Device: {device}")
    print(f"  • Tipos disponíveis: {len(GRASSCLOVER_PROMPTS)}")
    
    # Gerar lote
    batch_results = generate_grassclover_batch(
        num_images=BATCH_SIZE,
        apply_postprocess=APPLY_POSTPROCESS,
        debug=True
    )
    
    # Exibir resultados
    if batch_results:
        print(f"\n📊 Estatísticas do Lote:")
        print(f"  • Total gerado: {len(batch_results)}/{BATCH_SIZE}")
        print(f"  • Taxa de sucesso: {len(batch_results)/BATCH_SIZE*100:.1f}%")
        
        # Contagem por tipo
        type_counts = {}
        for result in batch_results:
            prompt_key = result['metadata']['prompt_key']
            type_counts[prompt_key] = type_counts.get(prompt_key, 0) + 1
        
        print(f"  • Distribuição por tipo:")
        for prompt_key, count in type_counts.items():
            description = GRASSCLOVER_PROMPTS[prompt_key]['description']
            print(f"    - {description}: {count} imagens")
        
        # Exibir grid
        print(f"\n🖼️  Exibindo resultados...")
        display_batch_results(batch_results)
        
        print(f"\n✅ Lote concluído com sucesso!")
        
    else:
        print(f"❌ Nenhuma imagem foi gerada no lote!")
        
else:
    print("❌ Pipeline não está pronto - não é possível gerar lote")

print("=" * 60)

## 💾 Salvamento das Imagens

Salva as imagens geradas com metadados organizados.

In [ ]:
# 💾 SALVAMENTO ORGANIZADO DAS IMAGENS
print("💾 Configurando sistema de salvamento...\n")

import os
import json

def save_grassclover_batch(batch_results, output_dir="grassclover_generated", save_metadata=True):
    """
    Salva lote de imagens com organização e metadados
    
    Args:
        batch_results: Lista de resultados da geração
        output_dir: Diretório de saída
        save_metadata: Salvar arquivos JSON com metadados
        
    Returns:
        dict com estatísticas de salvamento
    """
    
    if not batch_results:
        print("❌ Nenhuma imagem para salvar!")
        return {'success': False, 'saved_count': 0}
    
    try:
        # Criar diretórios
        os.makedirs(output_dir, exist_ok=True)
        images_dir = os.path.join(output_dir, "images")
        metadata_dir = os.path.join(output_dir, "metadata")
        
        os.makedirs(images_dir, exist_ok=True)
        if save_metadata:
            os.makedirs(metadata_dir, exist_ok=True)
        
        print(f"📁 Diretórios criados:")
        print(f"  • Principal: {output_dir}")
        print(f"  • Imagens: {images_dir}")
        if save_metadata:
            print(f"  • Metadados: {metadata_dir}")
        
        saved_count = 0
        saved_files = []
        
        # Salvar cada imagem
        for i, result in enumerate(batch_results):
            try:
                metadata = result['metadata']
                prompt_key = metadata['prompt_key']
                seed = metadata['seed']
                
                # Nome do arquivo
                filename_base = f"grassclover_{prompt_key}_{seed:06d}"
                
                # Salvar imagem original
                original_path = os.path.join(images_dir, f"{filename_base}_original.png")
                result['image'].save(original_path, 'PNG')
                
                files_saved = [original_path]
                
                # Salvar imagem processada se existir
                if 'processed_image' in result:
                    processed_path = os.path.join(images_dir, f"{filename_base}_processed.png")
                    result['processed_image'].save(processed_path, 'PNG')
                    files_saved.append(processed_path)
                
                # Salvar metadados
                if save_metadata:
                    metadata_path = os.path.join(metadata_dir, f"{filename_base}.json")
                    
                    # Preparar metadados para JSON (remover objetos não serializáveis)
                    json_metadata = metadata.copy()
                    json_metadata['files_saved'] = files_saved
                    json_metadata['save_timestamp'] = datetime.now().isoformat()
                    
                    with open(metadata_path, 'w', encoding='utf-8') as f:
                        json.dump(json_metadata, f, indent=2, ensure_ascii=False)
                    
                    files_saved.append(metadata_path)
                
                saved_files.extend(files_saved)
                saved_count += 1
                
                print(f"✅ Salvo {i+1}/{len(batch_results)}: {filename_base}")
                
            except Exception as e:
                print(f"❌ Erro ao salvar imagem {i+1}: {e}")
                continue
        
        # Criar índice geral
        if save_metadata:
            index_data = {
                'dataset_name': 'GrassClover Brazilian Synthetic',
                'generation_date': datetime.now().isoformat(),
                'total_images': len(batch_results),
                'saved_images': saved_count,
                'model_used': MODEL_ID,
                'device': str(device),
                'prompt_types': list(set(r['metadata']['prompt_key'] for r in batch_results)),
                'files': saved_files
            }
            
            index_path = os.path.join(output_dir, "dataset_index.json")
            with open(index_path, 'w', encoding='utf-8') as f:
                json.dump(index_data, f, indent=2, ensure_ascii=False)
            
            print(f"📋 Índice salvo: {index_path}")
        
        print(f"\n🎉 Salvamento concluído!")
        print(f"  • Imagens salvas: {saved_count}/{len(batch_results)}")
        print(f"  • Arquivos totais: {len(saved_files)}")
        print(f"  • Diretório: {os.path.abspath(output_dir)}")
        
        return {
            'success': True,
            'saved_count': saved_count,
            'total_files': len(saved_files),
            'output_dir': os.path.abspath(output_dir),
            'files': saved_files
        }
        
    except Exception as e:
        print(f"💥 Erro no salvamento: {e}")
        print(f"📋 DEBUG COPY/PASTE:")
        print(f"Output dir: {output_dir}")
        print(f"Batch results count: {len(batch_results) if batch_results else 0}")
        print(f"Error: {type(e).__name__}: {e}")
        
        return {
            'success': False,
            'saved_count': 0,
            'error': str(e)
        }

print("✅ Sistema de salvamento configurado!")
print("=" * 60)

In [ ]:
# 💾 SALVAR LOTE GERADO
print("💾 Salvando lote de imagens...\n")

if 'batch_results' in locals() and batch_results:
    # Configurações de salvamento
    OUTPUT_DIR = "grassclover_synthetic_dataset"
    SAVE_METADATA = True
    
    print(f"⚙️  Configurações de salvamento:")
    print(f"  • Diretório: {OUTPUT_DIR}")
    print(f"  • Salvar metadados: {SAVE_METADATA}")
    print(f"  • Imagens a salvar: {len(batch_results)}")
    
    # Executar salvamento
    save_result = save_grassclover_batch(
        batch_results=batch_results,
        output_dir=OUTPUT_DIR,
        save_metadata=SAVE_METADATA
    )
    
    # Resultado
    if save_result['success']:
        print(f"\n📊 Resumo do Salvamento:")
        print(f"  ✅ Sucesso: {save_result['saved_count']} imagens salvas")
        print(f"  📁 Localização: {save_result['output_dir']}")
        print(f"  📄 Arquivos totais: {save_result['total_files']}")
        
        # Listar estrutura de diretórios
        print(f"\n📋 Estrutura criada:")
        if os.path.exists(save_result['output_dir']):
            for root, dirs, files in os.walk(save_result['output_dir']):
                level = root.replace(save_result['output_dir'], '').count(os.sep)
                indent = ' ' * 2 * level
                print(f"{indent}{os.path.basename(root)}/")
                subindent = ' ' * 2 * (level + 1)
                for file in files[:3]:  # Mostrar só os primeiros 3 arquivos
                    print(f"{subindent}{file}")
                if len(files) > 3:
                    print(f"{subindent}... e mais {len(files)-3} arquivos")
        
        print(f"\n🎉 Dataset salvo com sucesso!")
        print(f"📂 Para acessar: {save_result['output_dir']}")
        
    else:
        print(f"❌ Falha no salvamento: {save_result.get('error', 'Erro desconhecido')}")
        
else:
    print("⚠️  Nenhum lote de imagens disponível para salvar")
    print("Execute primeiro a célula de geração em lote")

print("=" * 60)

## 📊 Relatório Final e Estatísticas

In [ ]:
# 📊 RELATÓRIO FINAL COMPLETO
print("📊 RELATÓRIO FINAL - GRASSCLOVER STABLE DIFFUSION\n")
print("=" * 70)

# Informações do sistema
print(f"🖥️  INFORMAÇÕES DO SISTEMA:")
print(f"  • Ambiente: {'Google Colab' if IN_COLAB else 'Local/Jupyter'}")
print(f"  • Device: {device if device else 'Não configurado'}")
print(f"  • PyTorch: {torch.__version__ if 'torch' in globals() else 'Não disponível'}")
print(f"  • Modelo SD: {MODEL_ID}")
print(f"  • Pipeline pronto: {'✅' if PIPELINE_READY else '❌'}")

if device and device.type == "cuda":
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    gpu_name = torch.cuda.get_device_name(0)
    print(f"  • GPU: {gpu_name} ({gpu_memory:.1f}GB)")

# Estatísticas de geração
print(f"\n🌾 ESTATÍSTICAS DE GERAÇÃO:")
if 'batch_results' in locals() and batch_results:
    print(f"  • Total de imagens geradas: {len(batch_results)}")
    print(f"  • Taxa de sucesso: 100%")
    
    # Contagem por tipo
    type_counts = {}
    seeds_used = []
    for result in batch_results:
        prompt_key = result['metadata']['prompt_key']
        type_counts[prompt_key] = type_counts.get(prompt_key, 0) + 1
        seeds_used.append(result['metadata']['seed'])
    
    print(f"  • Distribuição por tipo de pastagem:")
    for prompt_key, count in type_counts.items():
        description = GRASSCLOVER_PROMPTS[prompt_key]['description']
        percentage = (count / len(batch_results)) * 100
        print(f"    - {description}: {count} imagens ({percentage:.1f}%)")
    
    print(f"  • Seeds utilizadas: {min(seeds_used)} - {max(seeds_used)}")
    
    # Pós-processamento
    postprocessed = sum(1 for r in batch_results if r['metadata'].get('postprocessed', False))
    print(f"  • Imagens pós-processadas: {postprocessed}/{len(batch_results)}")
    
else:
    print(f"  • Nenhuma imagem foi gerada")

# Parâmetros utilizados
print(f"\n⚙️  PARÂMETROS DE GERAÇÃO:")
for param, value in GENERATION_PARAMS.items():
    print(f"  • {param}: {value}")

# Prompts disponíveis
print(f"\n📝 PROMPTS CONFIGURADOS ({len(GRASSCLOVER_PROMPTS)}):")
for prompt_key, prompt_data in GRASSCLOVER_PROMPTS.items():
    print(f"  • {prompt_key}: {prompt_data['description']}")

# Arquivos salvos
if 'save_result' in locals() and save_result.get('success', False):
    print(f"\n💾 ARQUIVOS SALVOS:")
    print(f"  • Diretório: {save_result['output_dir']}")
    print(f"  • Imagens salvas: {save_result['saved_count']}")
    print(f"  • Arquivos totais: {save_result['total_files']}")
    print(f"  • Inclui metadados: ✅")
    print(f"  • Índice do dataset: ✅")

# Próximos passos
print(f"\n🚀 PRÓXIMOS PASSOS SUGERIDOS:")
print(f"  1. 📸 Comparar com imagens reais do GrassClover Dataset")
print(f"  2. 🔧 Ajustar prompts baseado na qualidade visual")
print(f"  3. 📊 Gerar lotes maiores (50-100 imagens)")
print(f"  4. 🧠 Treinar modelo de segmentação com as imagens")
print(f"  5. 📈 Validar métricas (FID, LPIPS) contra dataset real")
print(f"  6. 🌾 Expandir para mais espécies (leguminosas, ervas daninhas)")

# Debugging info
print(f"\n🔧 DEBUG INFO (para troubleshooting):")
print(f"  • Timestamp: {datetime.now().isoformat()}")
print(f"  • Python: {sys.version.split()[0]}")
print(f"  • Platform: {platform.system()} {platform.release()}")
print(f"  • Working dir: {os.getcwd() if 'os' in globals() else 'Unknown'}")

print(f"\n{'='*70}")
print(f"🎉 NOTEBOOK EXECUTADO COM SUCESSO!")
print(f"🌾 Dataset GrassClover Brasileiro gerado usando Stable Diffusion")
print(f"📅 {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print(f"{'='*70}")

## 💡 Instruções para Uso e Debugging

### 🚨 **Em caso de erro:**
1. **Copie a seção "DEBUG COPY/PASTE"** que aparece nos erros
2. **Cole aqui nos comentários** ou no chat para análise
3. **Inclua informações do sistema** (GPU, memória, etc.)

### 🔧 **Ajustes possíveis:**
- **GENERATION_PARAMS**: Modifique `num_inference_steps` (15-50) para balancear qualidade/velocidade
- **BATCH_SIZE**: Reduza se houver problemas de memória
- **Seeds**: Mude para explorar diferentes variações
- **Prompts**: Ajuste para obter estilos visuais específicos

### 📸 **Para adicionar referências GrassClover:**
1. **Upload das imagens** na seção de arquivos do Colab
2. **Modifique os prompts** baseado nas características visuais
3. **Ajuste pós-processamento** para aproximar do estilo original

### 🎯 **Métricas de qualidade:**
- **Cobertura densa**: Pastagens devem ter ≥80% de cobertura vegetal
- **Perspectiva top-down**: Vista aérea consistente
- **Textura realística**: Detalhes de folhas e solo visíveis
- **Diversidade**: Variação entre as imagens geradas

---

**🌾 Dataset GrassClover Brasileiro - Gerado com Stable Diffusion**  
Baseado na metodologia de Skovsen et al. (CVPR 2019) adaptada para gramíneas tropicais.